In [ ]:
# Date: 06.02.25
# Note: Got it working (swapping out GPT for Gem), with the exception of using 
# 'DealSelection' (which formats the response)

# The Price is Right

Today we'll build another piece of the puzzle: a ScanningAgent that looks for promising deals by subscribing to RSS feeds.

In [1]:
print('working?')

working?


In [2]:
# imports

import os
import json
from dotenv import load_dotenv
from openai import OpenAI
from agents.deals import ScrapedDeal, DealSelection

In [3]:
# Initialize and constants

load_dotenv()
#os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY', 'your-key-if-not-using-env')
MODEL = 'gpt-4o-mini'
openai = OpenAI()

In [5]:
# Initialize and constants

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
# MODEL = 'gpt-4o-mini'
# openai = OpenAI()

# ----- Replacing with Gem code -----
import os
import google.generativeai as genai

genai.configure(api_key= api_key)

# Create the model
generation_config = {
  "temperature": 1,
  "top_p": 0.95,
  "top_k": 40,
  "max_output_tokens": 8192,
  "response_mime_type": "text/plain",
  #"response_mime_type": "application/json",
  #"response_mime_type": DealSelection,

}

model = genai.GenerativeModel(model_name="gemini-1.5-flash",
  generation_config=generation_config,)

chat_session = model.start_chat(
  history=[  ])

message = "this is a test only"
response = chat_session.send_message(message)
print(response.text)

There might be a problem with your API key? Please visit the troubleshooting notebook!
Understood.  I'm ready for your test. Please proceed.



In [3]:
deals = ScrapedDeal.fetch(show_progress=True)

 20%|██        | 1/5 [01:20<05:20, 80.08s/it]c:\Users\Tim_S\Desktop\bt\AIEng\llm_engineering\week8\agents\deals.py:27: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  description = BeautifulSoup(description, 'html.parser').get_text()
100%|██████████| 5/5 [07:26<00:00, 89.25s/it]


In [4]:
len(deals)

NameError: name 'deals' is not defined

In [5]:
deals[44].describe()

"Title: Shelves and Shelving Deals at Lowe's: Up to 25% off + free shipping w/ $35\nDetails: We've pictured the Outopee 5-Tier Freestanding Shelving Unit for $118 ($30 off). Choose in-store pickup to dodge the $5.99 shipping fee, or get free shipping with orders of $35 or more by signing up to MyLowe's Rewards. (It's free to join.) Shop Now at Lowe's\nFeatures: \nURL: https://www.dealnews.com/Shelves-and-Shelving-Deals-at-Lowes-Up-to-25-off-free-shipping-w-35/21716861.html?iref=rss-c196"

In [6]:
system_prompt = """You identify and summarize the 5 most detailed deals from a list, by selecting deals that have the most detailed, high quality description and the most clear price.
Respond strictly in JSON with no explanation, using this format. You should provide the price as a number derived from the description. If the price of a deal isn't clear, do not include that deal in your response.
Most important is that you respond with the 5 deals that have the most detailed product description with price. It's not important to mention the terms of the deal; most important is a thorough description of the product.
Be careful with products that are described as "$XXX off" or "reduced by $XXX" - this isn't the actual price of the product. Only respond with products when you are highly confident about the price. 

{"deals": [
    {
        "product_description": "Your clearly expressed summary of the product in 4-5 sentences. Details of the item are much more important than why it's a good deal. Avoid mentioning discounts and coupons; focus on the item itself. There should be a paragpraph of text for each item you choose.",
        "price": 99.99,
        "url": "the url as provided"
    },
    ...
]}"""

In [7]:
user_prompt = """Respond with the most promising 5 deals from this list, selecting those which have the most detailed, high quality product description and a clear price.
Respond strictly in JSON, and only JSON. You should rephrase the description to be a summary of the product itself, not the terms of the deal.
Remember to respond with a paragraph of text in the product_description field for each of the 5 items that you select.
Be careful with products that are described as "$XXX off" or "reduced by $XXX" - this isn't the actual price of the product. Only respond with products when you are highly confident about the price. 

Deals:

"""
user_prompt += '\n\n'.join([deal.describe() for deal in deals])

In [8]:
print(user_prompt[:2000])

Respond with the most promising 5 deals from this list, selecting those which have the most detailed, high quality product description and a clear price.
Respond strictly in JSON, and only JSON. You should rephrase the description to be a summary of the product itself, not the terms of the deal.
Remember to respond with a paragraph of text in the product_description field for each of the 5 items that you select.
Be careful with products that are described as "$XXX off" or "reduced by $XXX" - this isn't the actual price of the product. Only respond with products when you are highly confident about the price. 

Deals:

Title: Jackery Explorer 3000 Pro 3,024Wh Portable Power Station + 100W Solar Panel 2-Pack for $1,600 + free shipping
Details: That's $899 off list. Buy Now at Costco
Features: Vast 3024wh Battery Capacity, 3000-watt/6000-watt AC Output  Super Fast 2.4 hours to Fully Charged with Wall Outlet AC Charging  Portable Design with Pull-Out Rod and Two Wheels, Moving It Around Eas

In [9]:
pr = system_prompt + user_prompt
print(pr)

You identify and summarize the 5 most detailed deals from a list, by selecting deals that have the most detailed, high quality description and the most clear price.
Respond strictly in JSON with no explanation, using this format. You should provide the price as a number derived from the description. If the price of a deal isn't clear, do not include that deal in your response.
Most important is that you respond with the 5 deals that have the most detailed product description with price. It's not important to mention the terms of the deal; most important is a thorough description of the product.
Be careful with products that are described as "$XXX off" or "reduced by $XXX" - this isn't the actual price of the product. Only respond with products when you are highly confident about the price. 

{"deals": [
    {
        "product_description": "Your clearly expressed summary of the product in 4-5 sentences. Details of the item are much more important than why it's a good deal. Avoid mentio

In [1]:
response = chat_session.send_message(pr)
# response = DealSelection
print(response.text)

NameError: name 'chat_session' is not defined

In [ ]:
def get_recommendations():
    completion = openai.beta.chat.completions.parse(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
      ],
        response_format=DealSelection
    )
    result = completion.choices[0].message.parsed
    return result

In [ ]:
result = get_recommendations()

In [12]:
result = response.text
len(result.deals)

AttributeError: 'str' object has no attribute 'deals'

In [ ]:
result.deals[1]

In [ ]:
from agents.scanner_agent import ScannerAgent

In [ ]:
agent = ScannerAgent()
result = agent.scan()

In [ ]:
result